In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003881216049194336
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 11.712352053861887
200000 21.202294633739587
300000 24.10484917379647
400000 23.491425093744123
500000 22.84312181130922
600000 22.28075485363628
700000 23.385450883919248
800000 23.393750727416784
900000 21.494907066550205
1000000 18.998255837574284
1100000 10.979754217632031
1200000 20.225374077357703
1300000 19.47270526442092
1400000 18.073240577041876
1500000 19.576957500112314
1600000 19.099894130217383
1700000 18.421980462362065
1800000 19.562626598667567
1900000 18.852736384289543
2000000 20.388482219550106
2100000 4.968949049372406
2200000 21.379209455624526
2300000 24.118662951792224
2400000 23.721265829176964
2500000 22.965502798994255
2600000 24.745020729951257
2700000 21.79736429267099
2800000 23.01037592465192
2900000 21.653468325597387
300000

29400000 17.750693179693663
29500000 16.496554135294552
29600000 16.619606310018067
29700000 11.773759893345227
29800000 11.20626436669225
29900000 16.62034247164029
30000000 16.44449821698745
30100000 16.110709862621956
30200000 18.850767295466078
30300000 16.875388588458957
30400000 18.90619112222901
30500000 11.498591797272367
30600000 14.336962328082176
30700000 18.673129566931046
30800000 17.50664526551318
30900000 19.05858679688072
31000000 9.352662734752675
31100000 17.19879249770615
31200000 18.0650914320656
31300000 19.065777404929573
31400000 17.662886909718296
31500000 16.814265179242774
31600000 17.26987811971455
31700000 16.431871372586077
31800000 5.835761037805043
31900000 17.31543910950902
32000000 15.861054709370828
32100000 17.855652158732997
32200000 16.543119660581084
32300000 18.939112591535654
32400000 17.539110339583143
32500000 16.771481765543676
32600000 10.104274968640684
32700000 17.24346586686404
32800000 18.769597932601382
32900000 17.96979373085157
3300000

59100000 14.976548009729385
59200000 16.564989515946266
59300000 15.438740988303842
59400000 16.605781738059928
59500000 15.957765560272957
59600000 16.755838570038424
59700000 15.694419708005634
59800000 16.239942526793097
59900000 8.489176050462854
60000000 12.832819745022306
60100000 14.828142575507318
60200000 14.240073164360641
60300000 14.659333698951109
60400000 12.687485893609304
60500000 6.672038795498473
60600000 15.083947614348373
60700000 13.826154038309033
60800000 14.052429654276233
60900000 13.683239083401466
61000000 8.075195520645934
61100000 14.72402751215019
61200000 15.190699465269311
61300000 15.981951968691728
61400000 15.670340085540928
61500000 16.36835112165772
61600000 15.830460688592082
61700000 14.968079216945721
61800000 16.12314201099836
61900000 15.005199872337336
62000000 7.32877804282778
62100000 13.628718688064408
62200000 13.490273693451504
62300000 14.349812450819691
62400000 13.410186838048299
62500000 6.630525967937143
62600000 13.678185875430334
6

88800000 7.2842016411371
88900000 10.270197252398438
89000000 11.948606044248503
89100000 9.323441646106243
89200000 8.527971096201318
89300000 12.309557302741695
89400000 11.843153336150841
89500000 6.690001568032499
89600000 11.495936931430089
89700000 12.591407906095466
89800000 11.70048623299839
89900000 13.310179960589776
90000000 12.949674788140918
90100000 13.535095259116094
90200000 12.474296306676932
90300000 12.708255456564505
90400000 12.874860349381512
90500000 12.098416969344665
90600000 12.530781600072027
90700000 11.220074535637975
90800000 12.178281168748299
90900000 4.761040149459513
91000000 11.18087740845006
91100000 12.029263532575769
91200000 3.8655224251767994
91300000 11.210685227938342
91400000 11.513183521435787
91500000 5.826382446961956
91600000 11.57679126120605
91700000 11.566632216833947
91800000 11.437431313751798
91900000 11.628294088965887
92000000 12.774173457374022
92100000 12.373717638177663
92200000 13.255794010696906
92300000 12.459534734773058
924

117900000 3.9434625619207893
118000000 8.630698451537727
118100000 10.321301412654545
118200000 9.501702408193315
118300000 9.603909206836336
118400000 10.477625517965139
118500000 9.572374493487274
118600000 10.526447978527516
118700000 9.811956197967442
118800000 11.099982096097401
118900000 10.550774571314516
119000000 9.366497356172568
119100000 10.172027408114026
119200000 9.024289417213541
119300000 9.810076277344496
119400000 9.034929196471326
119500000 9.16070235143075
119600000 9.493617495524207
119700000 9.097521887542609
119800000 3.0461091009459684
119900000 1.695506549560213
120000000 7.301660282239619
120100000 8.708304208662616
120200000 8.81189457337017
120300000 9.943926614339226
120400000 9.020773600470887
120500000 9.842752842712075
120600000 8.999952657988077
120700000 9.38312501051913
120800000 10.172100941355822
120900000 10.945414524210761
121000000 11.276261099348059
121100000 10.149534511282724
121200000 10.66024060003906
121300000 9.574395335688752
121400000 1

146800000 7.467772645345749
146900000 8.286453555381874
147000000 7.859357497359668
147100000 8.447405913973583
147200000 7.826259597805274
147300000 8.022466781721453
147400000 8.789223869841
147500000 8.3960914851795
147600000 9.175600807575167
147700000 7.987071003415064
147800000 8.237869232530205
147900000 7.6493132161921995
148000000 8.163981505503642
148100000 7.840743500592368
148200000 7.526837761540977
148300000 7.767459740613874
148400000 3.6598021972519135
148500000 6.036764973148988
148600000 5.623396799249098
148700000 3.3268543316217345
148800000 7.394613531826838
148900000 8.033590404760066
149000000 7.529289919828784
149100000 7.58283590138509
149200000 8.216344188212801
149300000 7.595243434790153
149400000 8.59898254608147
149500000 8.395429887857546
149600000 8.934281820380164
149700000 8.343104219225626
149800000 7.88516621347083
149900000 8.096726385084898
150000000 7.8337646833541
150100000 8.337638441813402
150200000 7.469805470668148
150300000 7.98128861271054


176100000 6.293436359341715
176200000 6.531965548211407
176300000 6.529803381470645
176400000 6.298375374005472
176500000 6.529139956315827
176600000 5.943807724352388
176700000 6.146179591009742
176800000 4.650316502598073
176900000 3.733040095781726
177000000 6.106165648504067
177100000 5.70500762314639
177200000 3.1304878779019933
177300000 5.233679392208387
177400000 5.574645937513858
177500000 5.988961739816628
177600000 2.609398486907126
177700000 5.7620558039928005
177800000 5.947661882611227
177900000 6.305628894335473
178000000 5.994656848860688
178100000 6.640728851356663
178200000 6.250350130728954
178300000 6.679176298917406
178400000 6.30227854624685
178500000 6.616052723805528
178600000 6.134049405364477
178700000 6.163223324849671
178800000 5.566519005147846
178900000 2.6744504889034504
179000000 6.097182096303532
179100000 5.711560488975026
179200000 5.5296581991171765
179300000 2.7426150492172576
179400000 5.5768655029328285
179500000 6.058969045851432
179600000 5.0469

205300000 2.0274476422197254
205400000 4.093216292969702
205500000 4.017147318747534
205600000 3.8359359437839475
205700000 3.8360620800692735
205800000 3.798764638830328
205900000 1.417064351558543
206000000 4.019614935188719
206100000 3.841599200638008
206200000 3.7964880777173033
206300000 3.87802690445857
206400000 4.08742779431194
206500000 1.7357210552446722
206600000 4.326180669346422
206700000 4.3285932068162625
206800000 4.300234959996366
206900000 4.293165272565764
207000000 4.343466846422764
207100000 4.2531069731182445
207200000 4.2917459474304716
207300000 2.8893331834609555
207400000 3.1110019332444128
207500000 4.00780380754025
207600000 3.9339697948015453
207700000 3.6841369404561886
207800000 3.7097002039282363
207900000 2.979197094771641
208000000 2.3109007707328835
208100000 3.7263260773598827
208200000 3.733874929060598
208300000 3.9185279379218207
208400000 3.9767494905525447
208500000 3.678655894818579
208600000 2.1587277323123746
208700000 4.173136454307105
20880

234100000 2.10165933136974
234200000 2.1624069278459626
234300000 1.9943050983904753
234400000 1.9533019132720464
234500000 1.9825036033864558
234600000 0.6956381763735335
234700000 1.9165486611233538
234800000 1.9765454229003845
234900000 2.019715230746032
235000000 2.069519056622153
235100000 2.0264879550328354
235200000 2.09017196334129
235300000 2.049425287799941
235400000 0.8767278919337153
235500000 2.129430751693542
235600000 2.104829800520357
235700000 2.061211189095163
235800000 0.8944470604668524
235900000 2.025256497754517
236000000 2.0024324936685027
236100000 1.9753009068754335
236200000 1.9008718720402857
236300000 1.9785037658079208
236400000 1.9043755663414457
236500000 1.8447427252739086
236600000 1.0420034709846657
236700000 1.48549103702099
236800000 1.8224856509564578
236900000 1.8684598252422453
237000000 1.9764025199904243
237100000 1.8977887308078734
237200000 1.9333251186462999
237300000 1.9752869857574562
237400000 1.7986043998777383
237500000 0.995560349839007

262100000 0.0009595450007617473
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 7.369532240695423e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
